<a href="https://colab.research.google.com/github/RohitPatilRRP/valueLabs/blob/master/valueLabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import nltk

In [0]:
url_train = 'https://raw.githubusercontent.com/RohitPatilRRP/valueLabs/master/Train.csv'
train = pd.read_csv(url_train)
train.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [0]:
url_test = 'https://raw.githubusercontent.com/RohitPatilRRP/valueLabs/master/Test.csv'
test = pd.read_csv(url_test)
test.head()

,question,answer_text
0,What 'S the main idea of the text ?,The lack of career -- based courses in US high...
1,"In the summer high season , Finland does nt se...",the sun is out at night
2,If you want to apply for Chinese Business Inte...,have to get confirmed at least twice
3,"That afternoon , the boy 's clothes were dry b...",nobody made room for him in the water .
4,Which of the following statements is NOT true ?,There are twelve countries in the World Wildli...


In [0]:
x_train , y_train = train[:,'question'	:'answer_text'] , train['distractor',:]

TypeError: ignored

test
